In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, Activation

Using TensorFlow backend.


In [2]:
with open('alice.txt', 'r') as f:
    texts = f.read().split('\n')
    texts = ' '.join(texts)

In [3]:
vocab = sorted(list(set([c for c in texts])))
n_vocab = len(vocab)
c2i = {c:i for i, c in enumerate(vocab)}
i2c = {i:c for i, c in enumerate(vocab)}
print('vocab_size', n_vocab)

vocab_size 86


In [4]:
# Generate training data
seq_len = 10
step = 1
input_chars, label_chars = [], []
# given 'this is a book'
# input: 'this is a boo'
# output: 'k'
for i in range(0, len(texts)-seq_len, step):
    input_chars.append(texts[i:i+seq_len])
    label_chars.append(texts[i+seq_len])

In [5]:
# vectorize
N = len(input_chars)
X = np.zeros((N, seq_len, n_vocab)) # one-hot representaiton
Y = np.zeros((N, n_vocab))
for i, x in enumerate(input_chars):
    for t, ch in enumerate(x):
        X[i, t, c2i[ch]] = 1
    Y[i, c2i[label_chars[i]]] = 1

In [6]:
# define model
hidden_size = 64
batch_size = 128
n_epoch = 40

char_inputs = Input(shape=(seq_len, n_vocab))
hidden = SimpleRNN(hidden_size, unroll=True)(char_inputs)
char_output = Dense(n_vocab, activation='softmax')(hidden)
model = Model(char_inputs, char_output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 86)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                9664      
_________________________________________________________________
dense_1 (Dense)              (None, 86)                5590      
Total params: 15,254
Trainable params: 15,254
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.fit(X, Y,
          batch_size=batch_size,
          epochs=n_epoch,
          validation_split=.2)

Train on 131045 samples, validate on 32762 samples
Epoch 1/40
131045/131045 [==============================] - 3s 21us/step - loss: 2.6632 - acc: 0.3024 - val_loss: 2.6195 - val_acc: 0.3280
Epoch 2/40
131045/131045 [==============================] - 3s 23us/step - loss: 2.1728 - acc: 0.4064 - val_loss: 2.4964 - val_acc: 0.3518
Epoch 3/40
131045/131045 [==============================] - 3s 21us/step - loss: 2.0270 - acc: 0.4396 - val_loss: 2.4510 - val_acc: 0.3709
Epoch 4/40
131045/131045 [==============================] - 2s 19us/step - loss: 1.9434 - acc: 0.4571 - val_loss: 2.4161 - val_acc: 0.3743
Epoch 5/40
131045/131045 [==============================] - 2s 19us/step - loss: 1.8841 - acc: 0.4705 - val_loss: 2.3885 - val_acc: 0.3839
Epoch 6/40
131045/131045 [==============================] - 3s 22us/step - loss: 1.8392 - acc: 0.4811 - val_loss: 2.3633 - val_acc: 0.3855
Epoch 7/40
131045/131045 [==============================] - 3s 20us/step - loss: 1.8024 - acc: 0.4912 - val_loss: 2

In [8]:
# Prediction
n_pred = 10
for i in range(n_pred):
    testX = X[i]
    testX = np.reshape(testX, (1, testX.shape[0], testX.shape[1]))
    pred = model.predict(testX)
    ypred = i2c[np.argmax(pred)]
    print('input:', input_chars[i])
    print('predict:', ypred)
    print('label:', label_chars[i])

input: ﻿Project G
predict: r
label: u
input: Project Gu
predict: t
label: t
input: roject Gut
predict: e
label: e
input: oject Gute
predict: n
label: n
input: ject Guten
predict: t
label: b
input: ect Gutenb
predict: l
label: e
input: ct Gutenbe
predict: r
label: r
input: t Gutenber
predict:  
label: g
input:  Gutenberg
predict: e
label: ’
input: Gutenberg’
predict: t
label: s
